In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy

import abtem
import ase

# from ase.lattice.spacegroup import crystal
from ase.spacegroup import crystal
from abtem.visualize import show_atoms
from ase.io import read
from ase.visualize import view
from ase import Atoms
from abtem.parametrizations import LobatoParametrization
import plotly.express as px
import itertools

from mp_api.client import MPRester

In [7]:
def structure_from_MP(material_id , API_key = "cxnJrpd5pqO3S94GGmwCc3mlqtwYG9Yo", only_ase_Structure = True, conventional_unit_cell = False):
    #connecting to MP database with API key
    mpr = MPRester(API_key)
    
    # Fetch the structure for the material
    structure = mpr.get_structure_by_material_id(material_id, conventional_unit_cell = conventional_unit_cell)
    
    #retriveing the coorniates and species of atoms
    coordinates = []
    coordinates_frac = []
    species = []
    species_name = []
    structure_abc = np.array(structure.lattice.abc)
    structure_angles = np.array(structure.lattice.angles)
    alpha, beta, gamma = np.deg2rad(structure_angles[0]), np.deg2rad(structure_angles[1]), np.deg2rad(structure_angles[2])
    #calculating the volume:
    V = structure_abc[0]*structure_abc[1]*structure_abc[2]*np.sqrt(1-np.cos(alpha)**2-np.cos(beta)**2
        -np.cos(gamma)**2+2*np.cos(alpha)*np.cos(beta)*np.cos(gamma))
    
    #making the matrix to convert vectors from fractional coordinates to cartesian coordinates:
    A = np.transpose([[structure_abc[0],
          0,
          0],
           [structure_abc[1]*np.cos(gamma),
            structure_abc[1]*np.sin(gamma),
            0],
           [structure_abc[2]*np.cos(beta),
             structure_abc[2]*(np.cos(alpha)-np.cos(beta)*np.cos(gamma))/np.sin(gamma), 
            V/(structure_abc[0]*structure_abc[1]*np.sin(gamma)) ]
         ])
    
    for s in structure:
            coordinates_frac.append(s.frac_coords) #would give fractional coordinates instead
            coordinates.append(np.dot(A,np.transpose(s.frac_coords))) #cartesian coordinates calculated from fractional coordinates
            species.append(s.specie.Z) #atomic number
            species_name.append(s.specie) #would give strings (e.g. "Fe") instead of atomic number
    
    #saving data as dictionary
    lattice = {'structure':{'Name' : species_name, 'Atmoic Number' : species , 'Coordniates' : coordinates ,
              'Fractional coordinates' : coordinates_frac}, 'vector' : {'distance' : structure_abc, 'angles': structure_angles }}
    
    #making the unit cell compact form for ase (a,b,c,alpha,beta,gamma)
    cell = np.copy(np.append(lattice['vector']['distance'],lattice['vector']['angles']))
    ase_structure = Atoms(species, coordinates, cell = cell) #creating the strucutre in ase
    if only_ase_Structure:
        return ase_structure #returning only ase structure object
    else:
        return Atoms(species, coordinates, cell = cell), cell #returning ase structure object as well as cell dictionary


In [8]:
def get_covalent_radii(ase_structre):
    atomic_number = ase_structre.get_atomic_numbers() #retriving atomic number of each atom

    #list to store covalent radii
    covalent_radii = []
    for at_num in atomic_number:
        covalent_radii.append(ase.data.covalent_radii[at_num])
    return covalent_radii

In [11]:
def get_atom_2d_positions(ase_structure, round = 3):
    return np.round(ase_structure.get_positions()[:,0:2], 3)



In [12]:
def rotate_points(points, angle_degrees):
    """
    Rotate an array of 2D points around the origin by a given angle.

    Parameters:
    points (np.ndarray): An array of shape (n, 2) representing n 2D points.
    angle_degrees (float): The angle by which to rotate the points, in degrees.

    Returns:
    np.ndarray: The rotated array of 2D points.
    """
    # Convert the angle from degrees to radians
    angle_radians = np.radians(angle_degrees)

    # Define the rotation matrix https://en.wikipedia.org/wiki/Rotation_matrix
    rotation_matrix = np.array([
        [np.cos(angle_radians), -np.sin(angle_radians)],
        [np.sin(angle_radians), np.cos(angle_radians)]
    ])

    # Rotate the points
    rotated_points = np.dot(points, rotation_matrix)

    return rotated_points

In [13]:
def center_ase(ase_structure):
    #returns ases structure with center at the center of the grid
    coord = ase_structure.cell
    ase_structure.translate([-coord[0][0]/2, -coord[1][1]/2, 0])

In [14]:
def  generate_plotdata(ase_structure, theta):
    """"Function for generation plotable data for Moire pattern recognition
    ase_structure = an ase atoms object in the desired dimensions
    theta = a list angles to have plot of"""    

    pos = get_atom_2d_positions(ase_structure) #get the 2D coordinates of the atoms object

    #Defining lists used to generate dataframe
    angles_long = []
    radius_long = []
    species_long = []
    x_long = []
    y_long = []
    layer_long = []
    atom_index_long = []

    #define list of constant terms for variation in angle

    species = np.array(ase_structure.symbols) #get the symbols of the species
    radius = get_covalent_radii(ase_structure) #get the atomic radius of each species
    layer_1 = ['fixed layer']*len(pos) #list of layer labels
    layer_2 = ['twist layer']*len(pos)
    atom_index_1 = np.arange(1,len(pos)+1) #list of atom index (used for plotly identification of what variable should be animated)
    atom_index_2 = np.arange(len(pos)+1,len(pos)*2+1)
    

    for a in theta:
        a_s = [a]*len(pos) #making the angle array of the same length as the number of atoms
        rot_pos = rotate_points(pos, a) #rotating the atoms
        #appending the data to the lists
        angles_long.append(a_s)
        angles_long.append(a_s)
        x_long.append(pos[:,0])
        x_long.append(rot_pos[:,0])
        y_long.append(pos[:,1])
        y_long.append(rot_pos[:,1])
        species_long.append(species)
        species_long.append(species)
        radius_long.append(radius)
        radius_long.append(radius)
        layer_long.append(layer_1)
        layer_long.append(layer_2)
        atom_index_long.append(atom_index_1)
        atom_index_long.append(atom_index_2)
        

    angles_long = [item for sublist in angles_long for item in sublist]
    x_long = [item for sublist in x_long for item in sublist]
    y_long = [item for sublist in y_long for item in sublist]
    species_long = [item for sublist in species_long for item in sublist]
    radius_long = [item for sublist in radius_long for item in sublist]
    layer_long = [item for sublist in layer_long for item in sublist]
    atom_index_long = [item for sublist in atom_index_long for item in sublist]


    return pd.DataFrame({'angle': angles_long , 'x': x_long, 'y':y_long, 
                                'species':species_long, 'radius': radius_long, 
                                'layer': layer_long, 'atom_index': atom_index_long})   
     
    

# Make structure

In [15]:
material_Id = "mp-5229"
srtio3 = structure_from_MP(material_Id, conventional_unit_cell = False)
atoms = srtio3 * (20,20,1)
view(atoms)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

<Popen: returncode: None args: ['c:\\Users\\Bruger\\anaconda3\\envs\\moire_c...>

## Set angular range and resolution:

In [16]:
#defining the angular range and resolution
theta_max = 90
theta_min = 0
res=0.5
theta = np.arange(theta_min, theta_max+res, res)

## Generate plotting data

In [17]:
plotting_data = generate_plotdata(atoms, theta)

# Plotting the data

In [ ]:
#Standard plot
pos = atoms.positions
min_x = pos[:,0].min()/2
max_x = pos[:,0].max()/2
min_y = pos[:,1].min()/2
max_y = pos[:,1].max()/2
radius_max = plotting_data.radius.max()
Picture_size = 800
Pixel_fraction = 0.67 #fraction of pixels used for plot
size_reduction_factor = 0.5
Marker_size = radius_max/(max_x-min_x)*(Picture_size*Pixel_fraction*size_reduction_factor)

fig = px.scatter(plotting_data, x="x", y="y", animation_frame="angle", animation_group="atom_index",
          size="radius", size_max = Marker_size, color="species", hover_name="layer",
           range_x=[min_x, max_x], range_y=[min_y, max_y],
           color_discrete_sequence = ['green', 'grey', 'red'],
           width=Picture_size, height=Picture_size)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )
fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()

In [ ]:
#Black_wide_plot
min_x = pos[:,0].min()/2
max_x = pos[:,0].max()/2
min_y = pos[:,1].min()/2
max_y = pos[:,1].max()/2
radius_max = plotting_data.radius.max()
Picture_size = 800
Pixel_fraction = 0.67 #fraction of pixels used for plot
size_reduction_factor = 0.1
Marker_size = radius_max/(max_x-min_x)*(Picture_size*Pixel_fraction*size_reduction_factor)

fig = px.scatter(plotting_data, x="x", y="y", animation_frame="angle", animation_group="atom_index",
          size="radius", size_max = Marker_size, color="layer", hover_name="species",
           range_x=[min_x, max_x], range_y=[min_y, max_y],
           color_discrete_sequence = ['white'],
           width=Picture_size, height=Picture_size)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
    showgrid = False
  )
fig.update_xaxes(showgrid = False)
fig.update_layout(plot_bgcolor='black')
fig["layout"].pop("updatemenus") # optional, drop animation buttons
fig.show()